In [ ]:
# ENV["LD_LIBRARY_PATH"] = ""

# import Pkg
# Pkg.activate(".")

# pkgs = [
# # "BioSequences",
# # "DataFrames",
# # "Dates",
# # "DelimitedFiles",
# # "FASTX",
# # "GLM",
# # "HTTP",
# # "JSON",
# # "Graphs",
# # "MetaGraphs",
# # "MD5",
# # "Statistics",
# # "StatsPlots",
# # "uCSV",
# # "CodecZlib",
# # "YAML",
# "Revise",
# # "Kmers",
# # "StatsBase",
# # "ProgressMeter"
# ]
# Pkg.add(pkgs)
# for pkg in pkgs
#     eval(Meta.parse("import $pkg"))
# end

In [ ]:
data_directory = joinpath(dirname(pwd()), "data")
sample_directories = readdir(data_directory, join=true)
trim_galore_directories = filter(x -> isdir(x) && !occursin(".ipynb_checkpoints", x), reduce(vcat, readdir.(sample_directories, join=true)))

In [ ]:
# # QC filtered read sets
# for trim_galore_directory in trim_galore_directories
#     forward = first(filter(x -> occursin(r"_1_val_1.fq.gz",x), readdir(trim_galore_directory, join=true)))
#     reverse = first(filter(x -> occursin(r"_2_val_2.fq.gz",x), readdir(trim_galore_directory, join=true)))
#     out_dir = joinpath(trim_galore_directory, replace(basename(trim_galore_directory), "_trimgalore" => "_megahit"))

#     if isfile(joinpath(out_dir, "final.contigs.fa")) && isfile(joinpath(out_dir, "done"))
#         @show "$(out_dir) is done"
#     else
#         run(`sbatch 1.0.megahit.sh $(out_dir) $(forward) $(reverse)`)
#     end
# end

In [ ]:
# QC and human filtered read sets
for trim_galore_directory in trim_galore_directories
    forward = first(filter(x -> occursin(r"unmapped\.1\.fq\.gz", x), readdir(trim_galore_directory, join=true)))
    reverse = first(filter(x -> occursin(r"unmapped\.2\.fq\.gz", x), readdir(trim_galore_directory, join=true)))
    out_dir = replace(forward, ".1.fq.gz" => "") * "_megahit"
    if isfile(joinpath(out_dir, "final.contigs.fa")) && isfile(joinpath(out_dir, "done"))
        @show "$(out_dir) is done"
    else
        if isdir(out_dir)
            rm(out_dir, recursive=true)
        end
        run(`sbatch 1.0.megahit.sh $(out_dir) $(forward) $(reverse)`)
        # print(`sbatch 1.0.megahit.sh $(out_dir) $(forward) $(reverse)`)
    end
end

In [ ]:
# isdir(out_dir) && rm(out_dir, recursive=true)

In [ ]:
# run(`conda create -n megahit -c bioconda megahit`)
# ncores = 2
# run(`conda run --live-stream --no-capture-output -n megahit megahit --num-cpu-threads $(ncores) -1 $(forward) -2 $(reverse) -o $(out_dir)`)

# ncores = 2



In [ ]:
#     initial_assembled_fasta = "$(out_dir)/final.contigs.fa"
#     assembled_fastg = replace(initial_assembled_fasta, ".fa" => ".fastg")

#     if !isfile(initial_assembled_fasta)
#         run(`megahit --num-cpu-threads $(n_cores) -1 $trimmed_forward_reads -2 $trimmed_reverse_reads -o $(out_dir)`)
#         # run(`conda run --live-stream --no-capture-output -n viral-pangenome-discovery megahit --num-cpu-threads $(n_cores) -1 $trimmed_forward_reads -2 $trimmed_reverse_reads -o $(out_dir)`)
#     end

In [ ]:


#     # read in the assembled fasta file and parse contig identifiers to get final k length
#     final_k_lengths = unique([replace(first(split(FASTX.identifier(record), '_')), r"^k" => "") for record in Mycelia.open_fastx(initial_assembled_fasta)])
#     @assert length(final_k_lengths) == 1
#     final_k_length = parse(Int, first(final_k_lengths))
#     if !isfile(assembled_fastg)
#         run(pipeline(`megahit_toolkit contig2fastg $(final_k_length) $(initial_assembled_fasta)`, assembled_fastg))
#     end

#     assembled_gfa = "$(assembled_fastg).gfa"
#     if !isfile(assembled_gfa)
#         run(`Bandage reduce $(assembled_fastg) $(assembled_gfa)`)
#     end

#     assembled_fasta = assembled_gfa * ".fna"
#     if !isfile(assembled_fasta)
#         open(assembled_fasta, "w") do io
#             fastx_io = FASTX.FASTA.Writer(io)
#             gfa_graph = Mycelia.parse_gfa(assembled_gfa)
#             for v in Graphs.vertices(gfa_graph)
#                 record = FASTX.FASTA.Record(gfa_graph.vprops[v][:identifier], gfa_graph.vprops[v][:sequence])
#                 write(fastx_io, record)
#             end
#             close(fastx_io)
#         end
#     end

#     # generate a bandage plot of the assembly graph
#     bandage_outfile = "$(assembled_gfa).bandage.jpg"
#     if !isfile(bandage_outfile)
#         run(`Bandage image $(assembled_gfa) $bandage_outfile`)
#     end

#     # map reads to the assembly and run qualimap QC
#     bwt_index = "$(assembled_fasta).bwt"
#     if !isfile(bwt_index)
#         run(`bwa index $(assembled_fasta)`)
#     end

#     # mapped_reads_bam = "$(assembled_fasta).bwa.sorted.marked_duplicates.bam"
#     mapped_reads_bam = "$(assembled_fasta).bwa.bam"
#     # sorting and deduping
#     # if !isfile(mapped_reads_bam)
#     #     run(pipeline(
#     #         `bwa mem -R "@RG\tID:$(SRR)\tSM:bar" -t $(n_cores) $(assembled_fasta) $(trimmed_forward_reads) $(trimmed_reverse_reads)`,
#     #         `samtools collate --threads $(n_cores) -O - -`,
#     #         `samtools fixmate --threads $(n_cores) -m - -`,
#     #         `samtools sort --threads $(n_cores)`,
#     #         `samtools markdup --threads $(n_cores) - -`,
#     #         `samtools view --threads $(n_cores) -buh`,
#     #         mapped_reads_bams))
#     # end
#     # sorting - 50Gb - 1594.823 sec
#     if !isfile(mapped_reads_bam) || (filesize(mapped_reads_bam) == 0)
#         run(pipeline(
#             `bwa mem -R "@RG\tID:$(SRR)\tSM:bar" -t $(n_cores) $(assembled_fasta) $(trimmed_forward_reads) $(trimmed_reverse_reads)`,
#             `samtools sort --threads $(n_cores) -u -m 100M`,
#             `samtools view --threads $(n_cores) -bh`,
#             mapped_reads_bam))
#     end

#     # if !isfile("$(mapped_reads_bam).bai")
#     #     run(`samtools index $(mapped_reads_bam)`)
#     # end

#     qualimap_report_pdf = "$(out_dir)/qualimap/report.pdf"
#     qualimap_report_txt = "$(out_dir)/qualimap/genome_results.txt"

#     if !isfile(qualimap_report_pdf) || !isfile(qualimap_report_txt)
#         run(`
#             conda run --live-stream -n viral-pangenome-discovery
#             qualimap bamqc
#             -nt $(n_cores)
#             -bam $(mapped_reads_bam)
#             -outdir $(out_dir)/qualimap
#             -outformat PDF:HTML
#             --output-genome-coverage $(mapped_reads_bam).genome_coverage.txt
#             --java-mem-size=4G
#             `)
#     end
# end

# # qualimap_contig_coverage_table = Mycelia.parse_qualimap_contig_coverage(qualimap_report_txt)
# # qualimap_contig_coverage_table[!, "% Mapped bases"] = round.(qualimap_contig_coverage_table[!, "Mapped bases"] ./ sum(qualimap_contig_coverage_table[!, "Mapped bases"]) .* 100, digits=3);